In [ ]:
!pip install pacmap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 26.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=552447 sha256=fc2093abf586586cb8047e1cbed70747c941c6d4df285811395000ae33834698
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy


In [ ]:
import os
import numpy as np
import cv2
from skimage.transform import resize
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import shuffle
from matplotlib.colors import to_rgba
from functions_EDX import *
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from skimage import exposure
from scipy.ndimage import gaussian_filter, uniform_filter
from skimage.filters import rank
from skimage.morphology import disk
from skimage.morphology import black_tophat, disk, white_tophat
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.kernel_approximation import Nystroem
import matplotlib.cm as cm
import pacmap
import seaborn as sns
from PIL import Image
from scipy.ndimage import uniform_filter, gaussian_filter
from scipy.spatial import cKDTree
from scipy.stats import norm

ModuleNotFoundError: No module named 'functions_EDX'

*Functions* for filtering

In [ ]:


def filter(image, k,s):
    min_value = np.min(image)
    max_value = np.max(image)

# Invert the array
    image = min_value + max_value - image
    local_mean = uniform_filter(image, size=k)
    local_var = uniform_filter(image**2, size=k) - local_mean**2
    local_std = np.sqrt(local_var)
    def local_gaussian_filter(image, local_mean, local_std):
        local_std = np.where(local_std > 0, local_std, 1)
        normalized_image = (image - local_mean) / local_std
        filtered_image = gaussian_filter(normalized_image, sigma=s)
        return (filtered_image * local_std + local_mean)

    filtered_image= local_gaussian_filter(image, local_mean, local_std)

    return filtered_image

def process_channel(image, sigma):
    image = filter(image, k=(4*sigma)+1, s=sigma)
    mp = image
    return mp

def filtering(base_image, sigma):
    n_images, base_image_rows, base_image_columns, base_image_channels = base_image.shape
    emp = np.zeros((n_images, base_image_rows, base_image_columns, base_image_channels))
    for n in range(n_images):

        results =(process_channel(base_image[n, :, :, channel], sigma) for channel in range(base_image_channels))

        for channel, result in enumerate(results):
            emp[n, :, :, channel] = result

    return emp





Blurring operation using filters 1 and 5

In [ ]:
feature_map_nystroem = Nystroem(kernel='rbf', n_components=250, random_state=1000, gamma=None)
transformed_data=[]
selected_samples=[]
spectrum_folder = r"/content/"

scaler= StandardScaler()
file_names = []
file_names1 = []
images =[]
nch=4000
global_sum = np.zeros((nch,))
global_sum_sq = np.zeros((nch,))

tmp = os.listdir(spectrum_folder)
for filename in tmp:
    if filename.endswith('.npz'):
        file_names.append(os.path.join(spectrum_folder,filename))

file_names = sorted(file_names)

tilenos= np.arange(0,1)
X_ZCA_rescaled=[]
nch=250
s=1024
s1=1024
n=1
en=len(tilenos)
gray_images=[]
images1=[]
# tile10=r"C:\Users\P301644\Documents\Data\Dataset2\haadf_png\10.png"
# image1 = np.array(Image.open(tile10))

n_filters=2
filter_size1= 1

filter_size2=5
for tile_idx in tilenos:

    spectrum_2D = np.load(file_names[tile_idx], allow_pickle=True)['spectrum_2D']
    spectrum_2D_n = np.reshape(spectrum_2D, (1024, 1024, nch))
    spectrum_2D_n1 = spectrum_2D_n[ 0:, 0:,:nch]

    norm_flat_spectrum_2D_n = np.reshape(spectrum_2D_n1, (s * s, nch))
    data=norm_flat_spectrum_2D_n.reshape(-1,nch)
    data_for_filtering =data.reshape(1, s, s, nch)[:,:,:,:]

    filtered_channels= filtering(data_for_filtering,filter_size1)

    filtered_channels2= filtering(data_for_filtering, filter_size2)

    spectrum_2D =np.concatenate((filtered_channels2.reshape(-1,nch), filtered_channels.reshape(-1,nch)), axis=-1)

    Y =np.asarray(spectrum_2D, dtype=np.float32)

    images.append(Y)


st=0
en=len(tilenos)
x=1024
nch=250 * n_filters

spectrum_forpca=scaler.fit_transform(np.array(images[st:en], dtype=np.float32).reshape(en*s*s,nch))

#Perform PCA ont he extracted spectral features
pca_final=PCA(.95)

from sklearn.kernel_approximation import Nystroem
feature_map_nystroem = Nystroem(kernel='rbf', n_components=500, random_state=1000, n_jobs=10)

transformed_data=feature_map_nystroem.fit_transform(spectrum_forpca.reshape(-1,spectrum_forpca.shape[1]))

pc_keep = .95
pca = PCA(pc_keep)
X= pca.fit_transform(transformed_data)

Mini batch kmeans

In [ ]:
#perform mini batch k-means by fitting on the smoothed spectra and predicting on the raw
X_partial  = np.load('X_partial.npy')
X_partial_orig= np.load('X_partial_orig.npy')
from sklearn.kernel_approximation import Nystroem
feature_map_nystroem = Nystroem(kernel='rbf', n_components=500, random_state=1000, n_jobs=10)

transformed_data=feature_map_nystroem.fit_transform(X_partial.reshape(-1,X_partial.shape[1]))
transformed_data_orig=feature_map_nystroem.fit_transform(X_partial_orig.reshape(-1,X_partial.shape[1]))
from sklearn.cluster import MiniBatchKMeans
mbkmeans = MiniBatchKMeans(n_clusters=10, batch_size=10000, random_state=1)
mbkmeans.fit(transformed_data)
labels = mbkmeans.predict(transformed_data)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


In [13]:
#perform mini batch k-means by fitting on the smoothed spectra and predicting on the raw
X_partial  = np.load(r'/content/transformed_data0_2.npy')
X_partial_fine  = np.load(r'/content/transformed_data0_1.npy')
from sklearn.cluster import MiniBatchKMeans
mbkmeans = MiniBatchKMeans(n_clusters=2, batch_size=10000, random_state=1)
mbkmeans.fit(X_partial)
labels = mbkmeans.predict(X_partial_fine)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


In [ ]:
labels=np.load('labels.npz.npy')

In [ ]:
print(labels)

[9 7 7 ... 2 0 0]


User interaction for cluster selection

In [11]:

import matplotlib.pyplot as plt
import numpy as np
import os
import ipywidgets as widgets
from IPython.display import display, clear_output



# Dummy path for spectrum_folder - Replace with your actual path
spectrum_folder = '/path/to/spectrum_folder'

# Dropdown to select which cluster to sub-cluster
cluster_dropdown = widgets.Dropdown(
    options=[i for i in range(10)],
    description='Cluster:',
    disabled=False,
)

# Textbox to enter the number of sub-clusters
subcluster_input = widgets.IntText(
    value=2,
    description='Sub-clusters:',
    disabled=False
)

# Button to confirm the selection
button = widgets.Button(
    description='Sub-cluster'
)

def display_selected_cluster(change):
    """ Display the image of the selected cluster. """
    cluster = cluster_dropdown.value
    with output:
        clear_output(wait=True)  # Clear previous output
        plt.figure()
        plt.imshow((labels == cluster).reshape(1,1024, 1024)[0])
        plt.title(f'Cluster {cluster}')
        plt.show()
# Output widget to display the image and save it
output = widgets.Output()

def on_button_click(b):
    image_cluster=np.zeros(1*1024*1024)
    cluster = cluster_dropdown.value

    sub_clusters = subcluster_input.value
    from sklearn.cluster import MiniBatchKMeans, SpectralClustering

    # Spec =SpectralClustering(n_clusters=sub_clusters, affinity='rbf', n_jobs=10)
    mbkmeans = MiniBatchKMeans(n_clusters=sub_clusters, batch_size=2000, random_state=1)
    mbkmeans.fit(X_partial[labels == cluster])
    image_cluster[labels==cluster]= mbkmeans.predict(X_partial_fine[labels == cluster])
    # Display and save the cluster image
    with output:

      clear_output(wait=True)
      for i in range(sub_clusters):
        plt.figure()
        mask = (labels == cluster)
        plt.imshow(( mask & (image_cluster == i)).reshape(1,1024, 1024)[0])
        plt.show()
        # plt.savefig(os.path.join(spectrum_folder, f'{cluster}_subclusters_{sub_clusters}.png'))
# Attach the handler to the dropdown value change
cluster_dropdown.observe(display_selected_cluster, names='value')
button.on_click(on_button_click)

# Display the widgets
display(cluster_dropdown, subcluster_input, button, output)







Dropdown(description='Cluster:', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), value=0)

IntText(value=2, description='Sub-clusters:')

Button(description='Sub-cluster', style=ButtonStyle())

Output()

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


In [ ]:

#MINI BATCH KMEANS 4 TILES

#perform mini batch k-means by fitting on the smoothed spectra and predicting on the raw
X_partial  = np.load('/content/X_partial3489.npy')
X_partial_orig= np.load('/content/X_partial_orig3489.npy')
from sklearn.kernel_approximation import Nystroem
from sklearn.cluster import MiniBatchKMeans
mbkmeans = MiniBatchKMeans(n_clusters=10, batch_size=10000, random_state=1)
mbkmeans.fit(X_partial)
labels = mbkmeans.predict(X_partial)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


In [19]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# Assuming some pre-existing global variables like X_partial_orig, labels, etc.

# Dummy cluster data
cluster_hierarchy = {}

# Dropdown to select the cluster to sub-cluster
cluster_dropdown = widgets.Dropdown(
    options=[i for i in range(10)],
    description='Cluster:',
    disabled=False,
)

# Textbox to input the number of sub-clusters
subcluster_input = widgets.IntText(
    value=2,
    description='Sub-clusters:',
    disabled=False
)

# Button to perform sub-clustering
button = widgets.Button(
    description='Sub-cluster'
)

# Button to go back to the previous level
back_button = widgets.Button(
    description='Back',
    disabled=True  # Initially disabled because there's no previous level yet
)

# Radio buttons to select the sub-cluster for further sub-clustering
subcluster_radio = widgets.RadioButtons(
    options=[],
    description='Sub-cluster:',
    disabled=True
)

# Output widget to display images and interaction
output = widgets.Output()

# Track the current level of clustering to facilitate recursive sub-clustering
current_level = {}

def display_selected_cluster(change):
    """Display the image of the selected cluster."""
    cluster = cluster_dropdown.value
    with output:
        clear_output(wait=True)
        plt.figure()
        plt.imshow((labels == cluster).reshape(1, 1024, 1024)[0], cmap='gray')
        plt.title(f'Cluster {cluster}')
        plt.show()

    # Reset level tracking and disable the back button
    current_level[cluster] = 0
    back_button.disabled = True
    subcluster_radio.options = []
    subcluster_radio.disabled = True

def on_button_click(b):
    """Perform sub-clustering and update the sub-cluster radio buttons."""
    global cluster_hierarchy, current_level

    cluster = cluster_dropdown.value
    sub_clusters = subcluster_input.value

    from sklearn.cluster import MiniBatchKMeans

    # Determine the current level for the selected cluster
    level = current_level.get(cluster, 0)

    # Perform sub-clustering
    mbkmeans = MiniBatchKMeans(n_clusters=sub_clusters, batch_size=2000, random_state=1)
    image_cluster = np.zeros(1 * 1024 * 1024)

    if level == 0:
        mask = (labels == cluster)
    else:
        mask = cluster_hierarchy[cluster][level - 1][selected_subcluster]

    image_cluster[mask] = mbkmeans.fit_predict(X_partial[mask])

    # Store the sub-cluster information in the hierarchy
    if cluster not in cluster_hierarchy:
        cluster_hierarchy[cluster] = {}
    cluster_hierarchy[cluster][level] = {}

    for i in range(sub_clusters):
        cluster_hierarchy[cluster][level][i] = mask & (image_cluster == i)

    # Update radio button options for sub-clustering
    subcluster_radio.options = [f'Sub-cluster {i}' for i in range(sub_clusters)]
    subcluster_radio.disabled = False

    # Update the current level and enable the back button
    current_level[cluster] = level + 1
    back_button.disabled = False

    # Display the sub-cluster images
    with output:
        clear_output(wait=True)
        for i in range(sub_clusters):
            plt.figure()
            plt.imshow(cluster_hierarchy[cluster][level][i].reshape(1, 1024, 1024)[0], cmap='gray')
            plt.title(f'Sub-cluster {i} of Cluster {cluster} (Level {level})')
            plt.show()

def on_subcluster_select(change):
    """Display the image of the selected sub-cluster for further sub-clustering."""
    global selected_subcluster

    selected_cluster = cluster_dropdown.value
    selected_subcluster = int(subcluster_radio.value.split()[-1])

    with output:
        clear_output(wait=True)
        level = current_level.get(selected_cluster, 0) - 1  # Previous level
        plt.figure()
        plt.imshow(cluster_hierarchy[selected_cluster][level][selected_subcluster].reshape(1, 1024, 1024)[0], cmap='gray')
        plt.title(f'Sub-cluster {selected_subcluster} of Cluster {selected_cluster} (Level {level})')
        plt.show()

def on_back_button_click(b):
    """Go back to the previous level of sub-clustering."""
    selected_cluster = cluster_dropdown.value
    current_level[selected_cluster] -= 1

    if current_level[selected_cluster] == 0:
        back_button.disabled = True  # Disable back button if at the root level

    level = current_level[selected_cluster] - 1  # Previous level
    if level < 0:
        return  # No level to go back to

    # Update radio button options for the previous level
    subcluster_radio.options = [f'Sub-cluster {i}' for i in cluster_hierarchy[selected_cluster][level].keys()]
    subcluster_radio.disabled = False

    # Display the sub-cluster images for the previous level
    with output:
        clear_output(wait=True)
        for i in cluster_hierarchy[selected_cluster][level].keys():
            plt.figure()
            plt.imshow(cluster_hierarchy[selected_cluster][level][i].reshape(1, 1024, 1024)[0], cmap='gray')
            plt.title(f'Sub-cluster {i} of Cluster {selected_cluster} (Level {level})')
            plt.show()

# Attach handlers
cluster_dropdown.observe(display_selected_cluster, names='value')
button.on_click(on_button_click)
back_button.on_click(on_back_button_click)
subcluster_radio.observe(on_subcluster_select, names='value')

# Display the widgets
display(cluster_dropdown, subcluster_input, button, back_button, subcluster_radio, output)


Dropdown(description='Cluster:', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), value=0)

IntText(value=2, description='Sub-clusters:')

Button(description='Sub-cluster', style=ButtonStyle())

Button(description='Back', disabled=True, style=ButtonStyle())

RadioButtons(description='Sub-cluster:', disabled=True, options=(), value=None)

...

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitl

AttributeError: 'NoneType' object has no attribute 'split'

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


AttributeError: 'NoneType' object has no attribute 'split'

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitl